In [15]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.models as models
from dataset import get_dataset
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [8]:
def get_model():
    
    model = models.resnet34(False)
    model.fc = nn.Linear(512,9)
    return model
state_dict = torch.load("results/checkpoints/sched_type_None.pth",map_location = device)

model = get_model()
model.load_state_dict(state_dict)
model.to(device)
print()

In [9]:
dataset = get_dataset()
test_loader =  DataLoader(dataset['test_set'], batch_size=512,shuffle = False, pin_memory = True,num_workers = 4)

In [12]:
def val(model,criterion,val_loader):
    
    total_acc,total_loss = 0,0
    model.eval()
    total,correct = 0,0
    incorrect_class = [0]*9
    with torch.no_grad():
        for x,y in val_loader:
            B = y.size(0)

            x = x.to(device)
            y = y.to(device)
            y = y.view(B).long()

            logits = model(x)
            preds = torch.argmax(logits,1)

            loss = criterion(logits,y)
            total_loss += (loss.item()/len(val_loader))
            
            total += B
            correct += (preds == y).sum().item()
            
            for i in range(B):
                if(preds[i] != y[i]):
                    incorrect_class[y[i]] +=1
            
    total_acc = correct/total
    return total_acc,incorrect_class

In [16]:
criterion = nn.CrossEntropyLoss()
acc,incorrect_class = val(model,criterion,test_loader)
print(f"Test Acc: {acc}")
print(f"Incorrect predictions per class: {incorrect_class}")

Test Acc: 0.7362116991643454
Incorrect predictions per class: [452, 0, 76, 107, 302, 248, 327, 254, 128]
